In [1]:
import numpy as np
from urllib.request import urlopen
import urllib
import matplotlib.pyplot as plt # Visuals
import seaborn as sns
import sklearn as skl
import pandas as pd

In [2]:
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'

In [3]:
np.set_printoptions(threshold=np.nan)

In [4]:
names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heartdisease']
heartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)

In [5]:
heartDisease.head()
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']
del heartDisease['oldpeak']
heartDisease = heartDisease.replace('?', np.nan)
heartDisease.dtypes
heartDisease.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'heartdisease'],
      dtype='object')

In [6]:
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator


#from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
model = BayesianModel([('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'), ('exang', 'trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')])

In [7]:
# Learing CPDs using Maximum Likelihood Estimators
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
#for cpd in model.get_cpds():
# print("CPD of {variable}:".format(variable=cpd.variable))
 # print(cpd)
print(model.get_cpds('age'))
print(model.get_cpds('chol'))
print(model.get_cpds('sex'))
model.get_independencies()

╒═════════╤════════════╕
│ age(28) │ 0.00383142 │
├─────────┼────────────┤
│ age(29) │ 0.00383142 │
├─────────┼────────────┤
│ age(30) │ 0.00383142 │
├─────────┼────────────┤
│ age(31) │ 0.00766284 │
├─────────┼────────────┤
│ age(32) │ 0.0153257  │
├─────────┼────────────┤
│ age(33) │ 0.00766284 │
├─────────┼────────────┤
│ age(34) │ 0.0153257  │
├─────────┼────────────┤
│ age(35) │ 0.0191571  │
├─────────┼────────────┤
│ age(36) │ 0.0191571  │
├─────────┼────────────┤
│ age(37) │ 0.0306513  │
├─────────┼────────────┤
│ age(38) │ 0.0191571  │
├─────────┼────────────┤
│ age(39) │ 0.0344828  │
├─────────┼────────────┤
│ age(40) │ 0.0191571  │
├─────────┼────────────┤
│ age(41) │ 0.0383142  │
├─────────┼────────────┤
│ age(42) │ 0.0268199  │
├─────────┼────────────┤
│ age(43) │ 0.0421456  │
├─────────┼────────────┤
│ age(44) │ 0.0268199  │
├─────────┼────────────┤
│ age(45) │ 0.0229885  │
├─────────┼────────────┤
│ age(46) │ 0.045977   │
├─────────┼────────────┤
│ age(47) │ 0.0344828  │


(age _|_ exang, sex)
(age _|_ thalach, restecg, chol | heartdisease)
(age _|_ exang | sex)
(age _|_ sex | exang)
(age _|_ exang, sex | fbs)
(age _|_ restecg, chol | thalach, heartdisease)
(age _|_ thalach, chol | restecg, heartdisease)
(age _|_ thalach, restecg, chol | heartdisease, sex)
(age _|_ thalach, restecg | chol, heartdisease)
(age _|_ thalach, restecg, chol | heartdisease, trestbps)
(age _|_ thalach, restecg, chol | exang, heartdisease)
(age _|_ thalach, restecg, chol | fbs, heartdisease)
(age _|_ exang | fbs, sex)
(age _|_ thalach, restecg, chol, heartdisease | fbs, trestbps)
(age _|_ sex | exang, fbs)
(age _|_ chol | thalach, restecg, heartdisease)
(age _|_ restecg, chol | thalach, heartdisease, sex)
(age _|_ restecg | thalach, chol, heartdisease)
(age _|_ restecg, chol | thalach, heartdisease, trestbps)
(age _|_ restecg, chol | thalach, exang, heartdisease)
(age _|_ restecg, chol | thalach, fbs, heartdisease)
(age _|_ restecg, chol, heartdisease | thalach, fbs, trestbps)
(a

In [8]:
# Doing exact inference using Variable Elimination
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

In [9]:
# Computing the probability of bronc given smoke.
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 28})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5770 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4230 │
╘════════════════╧═════════════════════╛


In [10]:
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 100})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5510 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4490 │
╘════════════════╧═════════════════════╛
